# Data Cleaning:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from io import StringIO
import string
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer

In [2]:
with open(os.path.abspath('/content/drive/MyDrive/Datasets-Veltris/machinegenclf/subtaskB_train.jsonl'), 'rb') as f:
    train_data = pd.read_json(f, lines=True)
with open(os.path.abspath('/content/drive/MyDrive/Datasets-Veltris/machinegenclf/subtaskB_dev.jsonl'), 'rb') as f:
    dev_data = pd.read_json(f, lines=True)

In [12]:
import pandas as pd
import re

def remove_emojis_links(df, column_name):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    link_pattern = re.compile(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+")

    emojis_found = []
    links_found = []
    clean_column_name = column_name + '_clean'
    df[clean_column_name] = df[column_name]

    for index, row in df.iterrows():
        emojis = emoji_pattern.findall(row[column_name])
        if emojis:
            emojis_found.append((index, emojis))
            clean_text = emoji_pattern.sub('', row[column_name])
            df.at[index, clean_column_name] = clean_text

        links = link_pattern.findall(row[column_name])
        if links:
            links_found.append((index, links))
            clean_text = link_pattern.sub('', row[column_name])
            df.at[index, clean_column_name] = clean_text

    return emojis_found, links_found

emojis, links = remove_emojis_links(train_data, 'text')

In [10]:
emojis[:10]

[(73, ['☰']),
 (237, ['你好', 'こんにちは', '안녕하세요']),
 (546, ['♭', '♭', '♭', '♭', '♭', '♭', '♭', '♭']),
 (941, ['♭']),
 (966, ['⏩', '⏩']),
 (1381, ['♭']),
 (1467, ['☰', '☰', '☰']),
 (1512, ['✓', '✓']),
 (1957, ['☰', '☰']),
 (2237, ['早上好', '早', '안녕'])]

In [ ]:
print(train_data['text'].iloc[237])

In [8]:
!pip install langid

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langid: filename=langid-1.1.6-py3-none-any.whl size=1941172 sha256=31aae9cc5d2ee2d24f2fdca4768a0fb442b98be804611bde430d401ffc9107c5
  Stored in directory: /root/.cache/pip/wheels/23/c8/c6/eed80894918490a175677414d40bd7c851413bbe03d4856c3c
Successfully built langid


In [127]:
import pandas as pd
import re
import langid

def remove_http_strings(text):
    words = text.split()
    cleaned_words = [word for word in words if not word.startswith("http")]
    cleaned_text = ' '.join(cleaned_words)
    return cleaned_text

def remove_non_english(sentence):
    non_ascii_pattern = r'[^\x00-\x7F]+'
    clean_sentence = re.sub(non_ascii_pattern, '', sentence)

    return clean_sentence

def remove_emojis_links(df, column_name):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)

    link_pattern1 = re.compile(r"\bhttp[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+\b")
    link_pattern2 = re.compile(r'\b(?:https?://\S+|www\.\S+)\b')
    link_pattern3 = re.compile(r'\bwww\.[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,}\b')
    link_pattern4 = re.compile(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+")
    # non_english_pattern = re.compile(r'name=\w+;size=\d+', re.U)
    non_english_pattern = re.compile(u'[^\u0000-\u007F]+')

    rep_specichar_pattern = re.compile(r'(?:([^a-zA-Z0-9\s])\1{2,})+|\n{3,}')

    emojis_found = []
    links_found = []
    otherlangs_found = []
    rep_chars_found = []

    clean_column_name = column_name + '_clean'
    df[clean_column_name] = df[column_name]

    for index, row in df.iterrows():
      text = row[column_name]

      emojis = emoji_pattern.findall(text)
      if emojis:
          emojis_found.append((index, emojis))
          clean_text = emoji_pattern.sub('', text)
          text = clean_text

      links1 = link_pattern1.findall(text)
      if links1:
          links_found.append((index, links1))
          clean_text = link_pattern1.sub('', text)
          text = clean_text

      links2 = link_pattern2.findall(text)
      if links2:
          links_found.append((index, links2))
          clean_text = link_pattern2.sub('', text)
          text = clean_text

      links3 = link_pattern3.findall(text)
      if links3:
          links_found.append((index, links3))
          clean_text = link_pattern3.sub('', text)
          text = clean_text

      links4 = link_pattern4.findall(text)
      if links4:
          links_found.append((index, links4))
          clean_text = link_pattern3.sub('', text)
          text = clean_text

      other_lang = non_english_pattern.findall(text)
      if other_lang:
        otherlangs_found.append((index, other_lang))
        clean_text = non_english_pattern.sub('', text)
        text = clean_text

      rep_chars = rep_specichar_pattern.findall(text)
      if rep_chars:
        rep_chars_found.append((index, other_lang))
        clean_text = rep_specichar_pattern.sub('', text)
        text = clean_text

      text = remove_non_english(text)
      text = remove_http_strings(text)
      df.at[index, clean_column_name] = text

    return emojis_found, links_found, otherlangs_found, rep_chars_found


train_emojis, train_links, train_otherlang, train_repchars = remove_emojis_links(train_data, 'text')
dev_emojis, dev_links, dev_otherlang, dev_repchars = remove_emojis_links(dev_data, 'text')

In [130]:
print("train links len: ",len(train_links))
print("dev links len: ", len(dev_links))

train links len:  1748
dev links len:  64


#### checking if there are any other special chars/links:

In [131]:
import pandas as pd
import re

def remove_emojis_links(df, column_name):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    link_pattern = re.compile(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+")

    emojis_found = []
    links_found = []
    clean_column_name = column_name + '_clean'
    df[clean_column_name] = df[column_name]

    for index, row in df.iterrows():
        emojis = emoji_pattern.findall(row[column_name])
        if emojis:
            emojis_found.append((index, emojis))
            clean_text = emoji_pattern.sub('', row[column_name])
            df.at[index, clean_column_name] = clean_text

        links = link_pattern.findall(row[column_name])
        if links:
            links_found.append((index, links))
            clean_text = link_pattern.sub('', row[column_name])
            df.at[index, clean_column_name] = clean_text

    return emojis_found, links_found

train_emojis, train_links = remove_emojis_links(train_data, 'text_clean')
dev_emojis, dev_links = remove_emojis_links(dev_data, 'text_clean')

In [132]:
print(train_emojis)
print(train_links)
print(dev_links)
print(dev_emojis)

[]
[(8388, ['http://nraas.wikis']), (27708, ['http://_URL_5_']), (45683, ['http://en.wikipedia.org/wiki/Philippine_Underwater_Hockey_Confederation']), (66152, ['https://training.adobe.com/certification/exams.html'])]
[]
[]


In [133]:
print(len(train_links))
print(len(dev_links))

4
0


In [150]:
train_data.at[8388, 'text_clean'] = str(train_data.text_clean.iloc[8388]).replace('Modswikiathttp://nraas.wikis paces.com /MasterController.', '')
train_data.at[27708, 'text_clean'] = str(train_data.text_clean.iloc[27708]).replace('^1http://_URL_5_', '')
train_data.at[45683, 'text_clean'] = str(train_data.text_clean.iloc[45683]).replace('alsohttp://en.wikipedia.org/wiki/Philippine_Underwater_Hockey_Confederation', '')
train_data.at[66152, 'text_clean'] = str(train_data.text_clean.iloc[66152]).replace('tohttps://training.adobe.com/certification/exams.html#p=2.', '')

In [152]:
train_clean = train_data.drop(['text'], axis=1)
dev_clean = dev_data.drop(['text'], axis=1)

train_clean.rename(columns = {'text_clean':'text'}, inplace = True)
dev_clean.rename(columns = {'text_clean':'text'}, inplace = True)

with open(os.path.abspath('/content/drive/MyDrive/Datasets-Veltris/machinegenclf/subtaskB_train_clean.jsonl'), 'w') as f:
    f.write(train_clean.to_json(orient='records', lines=True, force_ascii=False))
with open(os.path.abspath('/content/drive/MyDrive/Datasets-Veltris/machinegenclf/subtaskB_dev_clean.jsonl'), 'w') as f:
    f.write(dev_clean.to_json(orient='records', lines=True, force_ascii=False))